# Perceptron and back propagation

## Some stuffs to begin with

In [1]:
import numpy as np

np.random.seed(0)

In [2]:
def glorot(size):
    limit = np.sqrt(6.0 / sum(size))
    return np.random.uniform(-limit, limit, size)


def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def sigmoid_prime(y):
    return y * (1 - y)


def relu(x):
    return x * (x >= 0)


def relu_prime(y):
    return 1 * (y >= 0)

## Setup

let $lr$ a constant in [0, 1]

let $x$, $y$ 2 vectors, where:
- $x$ is the input of dimension $(k, n)$
- $y$ is the expected output of dimension $(k, m)$

let $w$, $b$ 2 matrices, where
- $w$ is the weights of the layer of dimension $(n, m)$
- $b$ the bias of the layer of dimension $(1, m)$

In [3]:
lr = 0.001
w = glorot((2, 1))
b = np.zeros((1, 1))

x = np.array(
    [
        [0, 0],
        [0, 1],
        [1, 0],
        [1, 1],
    ]
)
y_true = np.array(
    [
        [0],
        [0],
        [0],
        [1],
    ]
)

## Forward pass

We will compute the forward and backward pass of a one dense layer with $n$ inputs and $m$ outputs with an activation function of type $relu$.

**Compute the linear transformation of $x$**

```math
\hat{z} = \hat{x} w + b
```

Then give some non linearity by applying the activation function

```math
\hat{y} = relu(\hat{z}), relu: x \mapsto xH(x)
```
where H is the Heaviside step function
```math
H(x) = \begin{cases}
   1 &\text{if } x >= 0 \\
   0 &\text{if } x < 0
\end{cases}
```

**Compute the $loss$ between the output $\hat{y}$ and the expected output $y$**

```math
loss(\hat{y}, y) = \frac{1}{2}(y - \hat{y})^2 \\
```

## Backward pass

We want to know how much we need to adjust our weights and biases of the layer to minimize the loss function. Basically we need to substract the gradients of the loss along w and b. It is basically a Newton algorithm to find a minima of the forward function minimizing the loss.

**Compute the gradient of the $loss$ along $w$**

```math
\tag{a} \nabla{loss(\hat{y}, y)} \ldotp w = \frac{\partial{loss(\hat{y}, y)}}{\partial{w}} 
```

Using the chain rules, we got

```math
\tag{b} \frac{\partial{loss(\hat{y}, y)}}{\partial{w}} = \frac{\partial{loss(\hat{y}, y)}}{\partial{\hat{y}}} \frac{\partial{\hat{y}}}{\partial{\hat{z}}} \frac{\partial{\hat{z}}}{\partial{w}}
```

where

```math
\tag{c}  \frac{\partial{loss(\hat{y}, y)}}{\partial{\hat{y}}} = -(y - \hat{y}) = \hat{y} - y
```
```math
\tag{d}  \frac{\partial{\hat{y}}}{\partial{\hat{z}}} = \frac{\partial{relu(\hat{z})}}{\partial{\hat{z}}} = \frac{\partial{(\hat{z}H(\hat{z}))}}{\partial{\hat{z}}} = 1 * \begin{cases}
   1 &\text{if } \hat{z} >= 0 \\
   0 &\text{if } \hat{z} < 0
\end{cases} = H(\hat{z})
```
```math
\tag{e}  \frac{\partial{(x w + b)}}{\partial{w}} = x^T
```

Combining all together:

```math
\boxed{\nabla{loss(\hat{y}, y)} \ldotp w =  x^T \otimes (\hat{y} - y) H(\hat{z})}
```

**Compute the gradient of the $loss$ along $b$**

```math
\tag{a} \nabla{loss(\hat{y}, y)} \ldotp b = \frac{\partial{loss(\hat{y}, y)}}{\partial{b}} 
```

Using the chain rules, we got

```math
\tag{b} \frac{\partial{loss(\hat{y}, y)}}{\partial{b}} = \frac{\partial{loss(\hat{y}, y)}}{\partial{\hat{y}}} \frac{\partial{\hat{y}}}{\partial{\hat{z}}} \frac{\partial{\hat{z}}}{\partial{b}}
```

where

```math
\tag{c}  \frac{\partial{loss(\hat{y}, y)}}{\partial{\hat{y}}} = -(y - \hat{y}) = \hat{y} - y
```
```math
\tag{d}  \frac{\partial{\hat{y}}}{\partial{\hat{z}}} = \frac{\partial{relu(\hat{z})}}{\partial{\hat{z}}} = \frac{\partial{(\hat{z}H(\hat{z}))}}{\partial{\hat{z}}} = 1 * \begin{cases}
   1 &\text{if } \hat{z} >= 0 \\
   0 &\text{if } \hat{z} < 0
\end{cases} = H(\hat{z})
```
```math
\tag{e}  \frac{\partial{(x w + b)}}{\partial{b}} = 1
```

Combining all together:

```math
\boxed{\nabla{loss(\hat{y}, y)} \ldotp b = (\hat{y} - y) H(\hat{z})}
```

**Update the w and b matrices with the respective gradients of the loss.**

```math
\boxed{
   \begin{aligned}
   w = w - lr * \nabla{loss(\hat{y}, y)} \ldotp w \\
   b = b - lr * \nabla{loss(\hat{y}, y)} \ldotp b
   \end{aligned}
}
```

Now, we need to pass the gradient of the loss to the next layer of the neural network and then perform the same calculation as above.

**Compute the gradient of the $loss$ along $\hat{x}$**

```math
\tag{a} \nabla{loss(\hat{y}, y)} \ldotp \hat{x} = \frac{\partial{loss(\hat{y}, y)}}{\partial{\hat{x}}} 
```

Using the chain rules, we got

```math
\tag{b} \frac{\partial{loss(\hat{y}, y)}}{\partial{\hat{x}}} = \frac{\partial{loss(\hat{y}, y)}}{\partial{\hat{y}}} \frac{\partial{\hat{y}}}{\partial{\hat{z}}} \frac{\partial{\hat{z}}}{\partial{\hat{x}}}
```

where

```math
\tag{c}  \frac{\partial{loss(\hat{y}, y)}}{\partial{\hat{y}}} = -(y - \hat{y}) = \hat{y} - y
```
```math
\tag{d}  \frac{\partial{\hat{y}}}{\partial{\hat{z}}} = \frac{\partial{relu(\hat{z})}}{\partial{\hat{z}}} = \frac{\partial{(\hat{z}H(\hat{z}))}}{\partial{\hat{z}}} = 1 * \begin{cases}
   1 &\text{if } \hat{z} >= 0 \\
   0 &\text{if } \hat{z} < 0
\end{cases} = H(\hat{z})
```
```math
\tag{e}  \frac{\partial{(x w + b)}}{\partial{x}} = w^T
```

Combining all together:

```math
\boxed{\nabla{loss(\hat{y}, y)} \ldotp \hat{x} = (\hat{y} - y) H(\hat{z}) \otimes w^T}
```

**Finally we can pass the gradient of the loss to the next layer (Hence gradient descent).**

In [4]:
for _ in range(5000):
    # Forward
    y_pred = relu(x @ w + b)

    # Compute loss
    loss = 0.5 * (y_true - y_pred) ** 2

    # Compute loss gradient along y_pred
    grad_loss = y_pred - y_true

    # Compute loss gradient along the function relu
    grad_loss = grad_loss * relu_prime(y_pred)

    # Compute the loss gradient along the weights and biases
    grad_w = x.T @ grad_loss
    grad_b = np.mean(grad_loss)

    # Compute the loss gradient along x, ready to pass the loss gradient to the next layer
    grad_loss = grad_loss @ w.T

    # Adjust weigts and biases
    w -= grad_w * lr
    b -= grad_b * lr

np.all(np.round(y_pred) == y_true)

True